<a href="https://colab.research.google.com/github/sepahpourka/Models/blob/main/Napier_40529878_Dissertation_TL_using_NNLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import regularizers
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train = 'https://raw.githubusercontent.com/sepahpourka/NSL-KDD-Datasets/main/KDDTrain%2B.txt'
test = 'https://raw.githubusercontent.com/sepahpourka/NSL-KDD-Datasets/main/KDDTest%2B.txt'
test21 = 'https://raw.githubusercontent.com/sepahpourka/NSL-KDD-Datasets/main/KDDTest-21.txt'

In [3]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

In [4]:
train_data = pd.read_csv(train,names=feature)
test_data = pd.read_csv(test,names=feature)
data = pd.concat([train_data, test_data], ignore_index=True)

data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
148513,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
148514,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back,15
148515,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [5]:
# Redistribute across common attack class
# Type of attacks: DoS (Denial of Service attacks), R2L (Root to Local attacks), U2R (User to Root attack) and Probe (Probing attacks).
def change_label(df):
  df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)
  df.label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

change_label(data)

In [6]:
# distribution of attack classes
data.label.value_counts()

normal    77054
Dos       53387
Probe     14077
R2L        3880
U2R         119
Name: label, dtype: int64

In [8]:
label = pd.DataFrame(data.label)


# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
from sklearn import preprocessing
le2 = preprocessing.LabelEncoder()
enc_label = label.apply(le2.fit_transform)
data['intrusion'] = enc_label
print(data.shape)
data

(148517, 44)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty,intrusion
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,4
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,4
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos,19,0
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,4
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal,21,4
148513,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal,21,4
148514,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,Dos,15,0
148515,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,4


In [9]:
df = pd.DataFrame(data)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty,intrusion
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,4
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,4
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos,19,0
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,4
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,4


In [10]:
df['information'] = df['duration'].astype(str) + " "+ df['protocol_type'].astype(str) + " "+ df['service'].astype(str) + " "+ df['flag'].astype(str) + " "+ df['src_bytes'].astype(str) + " "+ df['dst_bytes'].astype(str) + " "+ df['land'].astype(str) + " "+ df['wrong_fragment'].astype(str) + " "+ df['urgent'].astype(str) + " "+ df['hot'].astype(str) + " "+ df['num_failed_logins'].astype(str) + " "+ df['logged_in'].astype(str) + " "+ df['num_compromised'].astype(str) + " "+ df['root_shell'].astype(str) + " "+ df['su_attempted'].astype(str) + " "+ df['num_root'].astype(str) + " "+ df['num_file_creations'].astype(str) + " "+ df['num_shells'].astype(str) + " "+ df['num_access_files'].astype(str) + " "+ df['num_outbound_cmds'].astype(str) + " "+ df['is_host_login'].astype(str) + " "+ df['is_guest_login'].astype(str) + " "+ df['count'].astype(str) + " "+ df['srv_count'].astype(str) + " "+ df['serror_rate'].astype(str) + " "+ df['srv_serror_rate'].astype(str) + " "+ df['rerror_rate'].astype(str) + " "+ df['srv_rerror_rate'].astype(str) + " "+ df['same_srv_rate'].astype(str) + " "+ df['diff_srv_rate'].astype(str) + " "+ df['srv_diff_host_rate'].astype(str) + " "+ df['dst_host_count'].astype(str) + " "+ df['dst_host_srv_count'].astype(str) + " "+ df['dst_host_same_srv_rate'].astype(str) + " "+ df['dst_host_diff_srv_rate'].astype(str) + " "+ df['dst_host_same_src_port_rate'].astype(str) + " "+ df['dst_host_srv_diff_host_rate'].astype(str) + " "+ df['dst_host_serror_rate'].astype(str) + " "+ df['dst_host_srv_serror_rate'].astype(str) + " "+ df['dst_host_rerror_rate'].astype(str) + " "+ df['dst_host_srv_rerror_rate'].astype(str)


# remove attribute 'difficulty_level'
df.drop(['duration'],axis=1,inplace=True)
df.drop(['protocol_type'],axis=1,inplace=True)
df.drop(['service'],axis=1,inplace=True)
df.drop(['flag'],axis=1,inplace=True)
df.drop(['src_bytes'],axis=1,inplace=True)
df.drop(['dst_bytes'],axis=1,inplace=True)
df.drop(['land'],axis=1,inplace=True)
df.drop(['wrong_fragment'],axis=1,inplace=True)
df.drop(['urgent'],axis=1,inplace=True)
df.drop(['hot'],axis=1,inplace=True)
df.drop(['num_failed_logins'],axis=1,inplace=True)
df.drop(['logged_in'],axis=1,inplace=True)
df.drop(['num_compromised'],axis=1,inplace=True)
df.drop(['root_shell'],axis=1,inplace=True)
df.drop(['su_attempted'],axis=1,inplace=True)
df.drop(['num_root'],axis=1,inplace=True)
df.drop(['num_file_creations'],axis=1,inplace=True)
df.drop(['num_shells'],axis=1,inplace=True)
df.drop(['num_access_files'],axis=1,inplace=True)
df.drop(['num_outbound_cmds'],axis=1,inplace=True)
df.drop(['is_host_login'],axis=1,inplace=True)
df.drop(['is_guest_login'],axis=1,inplace=True)
df.drop(['count'],axis=1,inplace=True)
df.drop(['srv_count'],axis=1,inplace=True)
df.drop(['serror_rate'],axis=1,inplace=True)
df.drop(['srv_serror_rate'],axis=1,inplace=True)
df.drop(['rerror_rate'],axis=1,inplace=True)
df.drop(['srv_rerror_rate'],axis=1,inplace=True)
df.drop(['same_srv_rate'],axis=1,inplace=True)
df.drop(['diff_srv_rate'],axis=1,inplace=True)
df.drop(['srv_diff_host_rate'],axis=1,inplace=True)
df.drop(['dst_host_count'],axis=1,inplace=True)
df.drop(['dst_host_srv_count'],axis=1,inplace=True)
df.drop(['dst_host_same_srv_rate'],axis=1,inplace=True)
df.drop(['dst_host_diff_srv_rate'],axis=1,inplace=True)
df.drop(['dst_host_same_src_port_rate'],axis=1,inplace=True)
df.drop(['dst_host_srv_diff_host_rate'],axis=1,inplace=True)
df.drop(['dst_host_serror_rate'],axis=1,inplace=True)
df.drop(['dst_host_srv_serror_rate'],axis=1,inplace=True)
df.drop(['dst_host_rerror_rate'],axis=1,inplace=True)
df.drop(['dst_host_srv_rerror_rate'],axis=1,inplace=True)
df.drop(['difficulty'],axis=1,inplace=True)

df.head()

,label,intrusion,information
0,normal,4,0 tcp ftp_data SF 491 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,normal,4,0 udp other SF 146 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,Dos,0,0 tcp private S0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,normal,4,0 tcp http SF 232 8153 0 0 0 0 0 1 0 0 0 0 0 0...
4,normal,4,0 tcp http SF 199 420 0 0 0 0 0 1 0 0 0 0 0 0 ...


In [11]:
df

,label,intrusion,information
0,normal,4,0 tcp ftp_data SF 491 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,normal,4,0 udp other SF 146 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,Dos,0,0 tcp private S0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,normal,4,0 tcp http SF 232 8153 0 0 0 0 0 1 0 0 0 0 0 0...
4,normal,4,0 tcp http SF 199 420 0 0 0 0 0 1 0 0 0 0 0 0 ...
...,...,...,...
148512,normal,4,0 tcp smtp SF 794 333 0 0 0 0 0 1 0 0 0 0 0 0 ...
148513,normal,4,0 tcp http SF 317 938 0 0 0 0 0 1 0 0 0 0 0 0 ...
148514,Dos,0,0 tcp http SF 54540 8314 0 0 0 2 0 1 1 0 0 0 0...
148515,normal,4,0 udp domain_u SF 42 42 0 0 0 0 0 0 0 0 0 0 0 ...


In [12]:
df.shape

(148517, 3)

In [13]:
y_data= data['intrusion']
X_data= data['information']

In [14]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_data = labelencoder.fit_transform(y_data)
#from sklearn.preprocessing import LabelBinarizer
#y_data = LabelBinarizer().fit_transform(y_data)


y_data.shape

y_data

array([4, 4, 0, ..., 0, 4, 1])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, test_size=0.90, shuffle=True)
print('We can now split the the training data')
print('X_train has shape:',X_train.shape,'\nX_test has shape:',X_test.shape)

We can now split the the training data
X_train has shape: (14851,) 
X_test has shape: (133666,)


In [16]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import tensorflow_hub as hub
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2", input_shape=[], dtype=tf.string, trainable=True)

model = Sequential()
model.add(hub_layer)


model.add(Dense(64, activation='relu'))
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=1,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 50)                3250      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 48197165 (183.86 MB)
Trainable params: 48197165 (183.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [18]:
# Convert training data to type float
X_train = X_train.astype(str)
y_train = y_train.astype(float)

history = model.fit(X_train, y_train, epochs=30, batch_size=500,validation_split=0.9)

Epoch 1/30
3/3 [==============================] - 7s 528ms/step - loss: 0.0000e+00 - accuracy: 0.0916 - precision: 0.6384 - recall: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0978 - val_precision: 0.6459 - val_recall: 1.0000
Epoch 2/30
3/3 [==============================] - 1s 304ms/step - loss: 0.0000e+00 - accuracy: 0.0916 - precision: 0.6384 - recall: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0978 - val_precision: 0.6459 - val_recall: 1.0000
Epoch 3/30
3/3 [==============================] - 1s 207ms/step - loss: 0.0000e+00 - accuracy: 0.0916 - precision: 0.6384 - recall: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0978 - val_precision: 0.6459 - val_recall: 1.0000
Epoch 4/30
3/3 [==============================] - 1s 306ms/step - loss: 0.0000e+00 - accuracy: 0.0916 - precision: 0.6384 - recall: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0978 - val_precision: 0.6459 - val_recall: 1.0000
Epoch 5/30
3/3 [==============================] - 1s 332ms/step - loss: 0.0000e+